In [ ]:
import os
import dotenv

dotenv.load_dotenv()
fireworks_api = os.getenv("FIREWORKS_API_KEY")
OPENAI_API = os.getenv("OPENAI")
PINECONE_API = os.getenv("PINECONE_API")

In [ ]:
import requests
from bs4 import BeautifulSoup
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_pinecone import PineconeVectorStore

# Extract text from Wikipedia article
# url = "https://en.wikipedia.org/wiki/Attempted_assassination_of_Donald_Trump"
url = "https://en.wikipedia.org/wiki/Manhattan_Project"
text = BeautifulSoup(requests.get(url).text, "html.parser").get_text(strip=True)

text_splitter = CharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=0,
    separator=" ",
)

# Split text into chunks
text_chunks = text_splitter.split_text(text)
docs = text_splitter.create_documents(text_chunks)

# Generate embeddings and upload to Pinecone
embeddings = OpenAIEmbeddings(model="text-embedding-3-large", api_key=OPENAI_API)
vectorstore = PineconeVectorStore(
    index_name="testllama-31", embedding=embeddings, pinecone_api_key=PINECONE_API
)
vectorstore.add_documents(docs)

In [ ]:
from langchain_fireworks import ChatFireworks

llm = ChatFireworks(
    model="accounts/fireworks/models/llama-v3p1-405b-instruct",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    fireworks_api_key=fireworks_api,
)

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


template = """
I am creating a YouTube video to demonstrate your proficiency in Retrieval-Augmented Generation (RAG). Please use the provided context to answer the questions accurately. Below is the context for you to reference:

{context}

Question: {question}

Answer Here:"""
prompt = PromptTemplate.from_template(template)

chain = (
    {
        "context": vectorstore.as_retriever() | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [10]:
chain.invoke("What was the role of the S-1 Committee in the Manhattan Project?")

'The S-1 Committee, also known as the Section One Committee of the National Defense Research Committee (NDRC), played a crucial role in the Manhattan Project. According to the context, the S-1 Committee was formed to investigate and recommend research and development of nuclear energy for military and post-war purposes. Specifically, the S-1 Committee meeting on December 18, 1941, recommended pursuing all five technologies, which was approved by Vannevar Bush, James B. Conant, and others. This led to the establishment of the Metallurgical Laboratory in early 1942 to study plutonium and reactors using graphite as a neutron moderator. In essence, the S-1 Committee helped guide the research and development of nuclear energy in the United States during World War II.'

In [ ]:
chain.invoke("Who were the key members of the S-1 Committee?")

In [11]:
chain.invoke("Who were the leading scientists involved in the Manhattan Project?")

'Based on the provided context, the leading scientists involved in the Manhattan Project included:\n\n1. Enrico Fermi\n2. Ernest Lawrence (not explicitly mentioned in the context, but known to be a key figure in the project)\n3. Emilio Segrè\n4. Richard Feynman\n5. Klaus Fuchs\n6. George Kistiakowsky\n7. Glenn Seaborg\n8. Leo Szilard\n9. Luis Alvarez\n10. Hans Bethe\n11. Niels Bohr\n12. Norris Bradbury\n13. James Chadwick\n14. John Cockcroft\n15. Enrico Fermi\n\nNote that this is not an exhaustive list, as the context provides a large number of names associated with the project. However, the above list includes some of the most notable and influential scientists involved in the Manhattan Project.'

In [ ]:
chain.invoke(
    "What were the initial recommendations of the S-1 Committee for the Manhattan Project?"
)

In [ ]:
chain.invoke("How did the Manhattan Project impact post-war science and technology?")